# seg_human

In [3]:
import sys,os
import configargparse

import torch
import numpy as np

from modeling import SRNsModel
import util

_LOG_ROOT = '/mnt/data/new_disk/liury/log/SRN/050816human_seg/'

_MODEL_PATH = os.path.join(_LOG_ROOT, 'checkpoints/epoch_0094_iter_050000.pth')

_TOT_NUM_INSTANCES = 184

model = SRNsModel(num_instances=_TOT_NUM_INSTANCES,
                  latent_dim=256,
                  use_unet_renderer=False,
                  tracing_steps=10,
                  freeze_networks=True,
                  out_channels=12,
                  img_sidelength=128,
                  output_sidelength=512
                 )

util.custom_load(model, path=_MODEL_PATH, discriminator=None,
                 overwrite_embeddings=False, overwrite_cam=True)

model.eval()
model.cuda()

ModuleNotFoundError: No module named 'modeling'

In [ ]:
from dataset.face_dataset import FaceRandomPoseDataset
from torch.utils.data import DataLoader
# from torch.utils.tensorboard import SummaryWriter

import cv2
import imageio

_OUTPUT_DIR = os.path.join(_LOG_ROOT, 'output_iter_020000')
_MODE = 'sphere'
_R = 1.5
_CAM_INT = '/data/anpei/facial-data/seg_human/intrinsics.txt'

_CAM_CENTER = [0.0, 1.6, 0.0]

_NUM_INSTANCES=10
_NUM_OBSERVATIONS=25

output_dir = os.path.join(_OUTPUT_DIR, _MODE+'_128')
sample_instances = list(np.random.choice(range(_TOT_NUM_INSTANCES), _NUM_INSTANCES, replace=False))

# writer = SummaryWriter(output_dir)

dataset = FaceRandomPoseDataset(
    intrinsics=_CAM_INT,
    cam_center=_CAM_CENTER,
    num_instances=sample_instances, 
    num_observations=_NUM_OBSERVATIONS, 
    sample_radius=_R, mode=_MODE)

dataloader = DataLoader(dataset,
                     collate_fn=dataset.collate_fn,
                     batch_size=1,
                     shuffle=False,
                     drop_last=False)

print('Beginning evaluation...')

images = {}

with torch.no_grad():

    for idx, model_input in enumerate(dataloader):
        model_input, ground_truth = model_input
        
        pose = model_input['pose']
        
#         print(pose)
        
        intrinsics = model_input['intrinsics']
        uv = model_input['uv']
        z = model.get_embedding(model_input)
        
        model_outputs = model(pose, z, intrinsics, uv)
        predictions, depth_maps = model_outputs
                
        batch_size, tensor_len, channels = predictions.shape
        img_sidelen = np.sqrt(tensor_len).astype(int)
                
        pred = torch.argmax(predictions, dim=2, keepdim=True)
        output_img = util.lin2img(pred, color_map=dataset.color_map).cpu().numpy()
        output_pred = pred.view(batch_size, img_sidelen, img_sidelen, 1).cpu().numpy()
#         output_dpt = util.lin2img(depth_maps).cpu().numpy()
#         output_dpt = (output_dpt - np.min(output_dpt)) / (np.max(output_dpt)-np.min(output_dpt) + 1e-5)
#         print(idx, 'depth = ', np.max(output_dpt), np.min(output_dpt), output_dpt.shape)
        
#         model.write_updates(writer, model_outputs, iter=idx, mode='test')

        for i in range(output_img.shape[0]):
            instance_idx = int(model_input['instance_idx'][i].squeeze().detach().cpu().numpy().astype(np.int64))
            observation_idx = model_input['observation_idx'][i]
            
            instance_dir = os.path.join(output_dir, "%03d" % instance_idx)
            os.makedirs(instance_dir, exist_ok=True)
            
            img = output_img[i, :, :, :].squeeze().transpose(1, 2, 0)
            img *= 255
            img = img.round().clip(0, 255).astype(np.uint8)
#             img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 

            if not instance_idx in images:
                images[instance_idx] = [None] * _NUM_OBSERVATIONS

            images[instance_idx][observation_idx] = img
#             output_fp = os.path.join(instance_dir, '%02d.png'%(observation_idx))
#             util.write_img(img, output_fp)
            
            output_fp = os.path.join(instance_dir, '%02d_seg.png'%(observation_idx))
            seg_img = output_pred[i, :, :].squeeze().astype(np.uint8)
            
            util.write_img(seg_img, output_fp)
            
#             dpt_img = (output_dpt[i, :, :, :].squeeze() * 255.0).astype(np.uint8)
#             dpt_img = cv2.applyColorMap(dpt_img, cv2.COLORMAP_JET)
#             output_fp = os.path.join(instance_dir, '%02d_depth.png'%(observation_idx))
# #             print('Save output for instance %03d - %02d: %s'%(instance_idx, observation_idx, output_fp))
#             util.write_img(dpt_img, output_fp)
            print('Save output for instance %04d - %02d: %s'%(instance_idx, observation_idx, output_fp))
        
            if observation_idx == _NUM_OBSERVATIONS - 1:
                imageio.mimsave(os.path.join(instance_dir, 'output.gif'), images[instance_idx], fps=5.0)
                print('=== [DONE] saving output.gif.')
            
#         print('[DONE] Save output for instance %03d.'%(instance_idx))


# seg face

In [2]:
import sys,os

import torch
import numpy as np

sys.path.append('..')

from modeling import SRNsModel
import util

_RENDERER = 'FC'

# _MODEL_PATH = '../checkpoints/epoch_0020_iter_090000.pth'
# _TOT_NUM_INSTANCES = 1494
# _OPT_CAM = False

# _MODEL_PATH = '../checkpoints/050701face_seg_2000_depth.pth'
# _OPT_CAM = False
# _TOT_NUM_INSTANCES = 3714

# _MODEL_PATH = '/root/liury/log/SRNs/060706face_celebA/checkpoints/epoch_1359_iter_170000.pth'
# _OPT_CAM = True
# _TOT_NUM_INSTANCES = 1000


# _MODEL_PATH = '/root/liury/log/SRNs/061905seg_teaser/checkpoints/epoch_2400_iter_012000.pth'
# _OPT_CAM = True
# _TOT_NUM_INSTANCES = 37


# _MODEL_PATH = '../checkpoints/061721face_seg_800.pth'
# _OPT_CAM = False
# _TOT_NUM_INSTANCES = 791

# _MODEL_PATH = '/home/anpei/liury/log/SRNs/062923face_seg_800/checkpoints/epoch_0036_iter_110000.pth'
# _OPT_CAM = False
# _TOT_NUM_INSTANCES = 711

_MODEL_PATH = '/home/anpei/liury/log/SRNs/070212face_seg_800/checkpoints/epoch_0010_iter_030000.pth'
_OPT_CAM = False
_TOT_NUM_INSTANCES = 664


# _MODEL_PATH = '/home/anpei/liury/log/SRNs/070821face_seg_800_imae/checkpoints/epoch_0007_iter_020000.pth'
# _OPT_CAM = False
# _TOT_NUM_INSTANCES = 664
# _RENDERER = 'ImAE'

# _MODEL_PATH = '/home/anpei/liury/log/SRNs/061701face_celebA/checkpoints/epoch_0840_iter_420000.pth'
# _OPT_CAM = True
# _TOT_NUM_INSTANCES = 4000

# _MODEL_PATH = os.path.join(
#     os.getenv("HOME"), 'liury/log/SRNs/061916face_celebA/checkpoints/epoch_2000_iter_014000.pth')
# _LOG_ROOT = os.path.join(
#     os.getenv("HOME"), 'liury/log/SRNs/061916face_celebA/reproj_celebA')
# os.makedirs(os.path.join(_LOG_ROOT, 'vis'), exist_ok=True)
# _OPT_CAM=True

# _TOT_NUM_INSTANCES = 50
# _TOT_NUM_INSTANCES = 3714

_IMG_SIZE = 128
_OUT_SIZE = 512

torch.cuda.set_device(0)

model = SRNsModel(num_instances=_TOT_NUM_INSTANCES,
                  latent_dim=256,
                  renderer=_RENDERER,
                  tracing_steps=10,
                  freeze_networks=True,
                  out_channels=20,
                  img_sidelength=_IMG_SIZE,
                  output_sidelength=_OUT_SIZE,
                  opt_cam=_OPT_CAM
                 )

util.custom_load(model, path=_MODEL_PATH, discriminator=None,
                 overwrite_embeddings=False, overwrite_cam=True)

model.eval()
model.cuda()


[INIT embedding] variable.
[INIT renderer] FC, with renderer = FC


SRNsModel(
  (latent_codes): Embedding(664, 256)
  (hyper_phi): HyperFC(
    (layers): ModuleList(
      (0): NewCls(
        (hyper_linear): HyperLinear(
          (hypo_params): FCBlock(
            (net): Sequential(
              (0): FCLayer(
                (net): Sequential(
                  (0): Linear(in_features=256, out_features=256, bias=True)
                  (1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
                  (2): ReLU(inplace=True)
                )
              )
              (1): FCLayer(
                (net): Sequential(
                  (0): Linear(in_features=256, out_features=256, bias=True)
                  (1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
                  (2): ReLU(inplace=True)
                )
              )
              (2): Linear(in_features=256, out_features=1024, bias=True)
            )
          )
        )
        (norm_nl): Sequential(
          (0): LayerNorm((256,), eps=1e-05, elementwise_

In [3]:
import numpy as np
from dataset.face_dataset import _campos2matrix
import cv2

from matplotlib import pyplot as plt

_CMAP = np.asarray([[0, 0, 0], [127, 212, 255], [255, 255, 127], [255, 255, 127], # 'background','skin', 'l_brow', 'r_brow'
                    [255, 255, 170], [255, 255, 170], [240, 157, 240], [255, 212, 255], #'l_eye', 'r_eye', 'r_nose', 'l_nose',
                    [31, 162, 230], [127, 255, 255], [127, 255, 255], #'mouth', 'u_lip', 'l_lip'
                    [0, 255, 85], [0, 255, 85], [0, 255, 170], [255, 255, 170], #'l_ear', 'r_ear', 'ear_r', 'eye_g'
                    [127, 170, 255], [85, 0, 255], [255, 170, 127], #'neck', 'neck_l', 'cloth'
                    [212, 127, 255], [0, 170, 255]#, 'hair', 'hat'
                    ])

_CMAP =torch.tensor(_CMAP, dtype=torch.float32) / 255.0

def _build_cam_int(focal, H, W):
    return np.array([  [focal, 0., W // 2, 0.],
                       [0., focal, H // 2, 0],
                       [0., 0, 1, 0],
                       [0, 0, 0, 1]])


def render_scene(model, pose, z, focal, img_sidelength):
    
    with torch.no_grad():
        pose = torch.from_numpy(pose).float().unsqueeze(0)
        cam_int = torch.from_numpy(
            _build_cam_int(focal, _IMG_SIZE, _IMG_SIZE)).float().unsqueeze(0)

        uv = np.mgrid[0:_IMG_SIZE, 0:_IMG_SIZE].astype(np.int32)
        uv = torch.from_numpy(np.flip(uv, axis=0).copy()).long()
        uv = uv.reshape(2, -1).transpose(1, 0).unsqueeze(0)

#         print(pose.shape, cam_int.shape, uv.shape, z.shape)

        predictions, depth_maps = model(pose, z, cam_int, uv)

        pred = torch.argmax(predictions, dim=2, keepdim=True)

#         print(pred.shape)

        out_img = util.lin2img(pred, color_map=_CMAP).cpu().numpy()
        out_seg = pred.view(img_sidelength, img_sidelength, 1).cpu().numpy()
        
        out_img = (out_img.squeeze().transpose(1, 2, 0)) * 255.0
        out_img = out_img.round().clip(0, 255).astype(np.uint8)

#         output_fp = os.path.join(instance_dir, '%02d_seg.png'%(observation_idx))
        out_seg = out_seg.squeeze().astype(np.uint8)

#         plt.subplot(1, 2, 1)
#         plt.imshow(out_img)
#         plt.subplot(1, 2, 2)
#         plt.imshow(out_seg)

#         plt.show()

        return out_img, out_seg
    

In [4]:
import numpy as np

import imageio

import matplotlib.image as mpimg

from dataset.face_dataset import _campos2matrix
from dataset import data_util

def render_custo_path(rener_mode, src_latent, init_dpt=0.8, trgt_latent=None, cam_center=np.array([0,0.11,0.0]), output_dir='output_custom', offset=0):
    
    _DEFAULT_CAM_INT = '../checkpoints/intrinsics.txt'
    cam_int = data_util.parse_intrinsics(_DEFAULT_CAM_INT, trgt_sidelength=128)
    focal = cam_int[0, 0]

    img_outputs = []
    seg_outputs = []

    idx = offset
    steps = 20
        
    def _get_latent(idx, tot_steps=steps, inv=False):
        if trgt_latent is None:
            return src_latent
        else:
            _w = idx / (steps-1)
            if inv:
                _w = (1.0 - _w)
            
            return (1.0-_w)*src_latent + _w*trgt_latent
      
    dpt = init_dpt
    cam_pose = _campos2matrix(np.array([0., 0., dpt])+cam_center, cam_center)
    
##################### fly in
#     print('*********************** FLY IN ***********************', idx)    

#     _range = np.asarray([-1.0, 1.0])*0.2 + init_dpt
#     dpts = np.concatenate([
#         np.linspace(init_dpt, _range[0], 5), 
#         np.linspace(_range[0], _range[1], 10),
#         np.linspace(_range[1], init_dpt, 5)], axis=0)

#     _range = 0.2
#     dpts = np.linspace(init_dpt, init_dpt-_range, steps)
    
#     for i, dpt in enumerate(dpts):
#         cam_pose = _campos2matrix(
#             np.array([0., 0., dpt])+cam_center,
#             cam_center)

#         out_img, out_seg = render_scene(model, cam_pose, _get_latent(i), focal, _OUT_SIZE)
#         img_outputs.append(out_img)
#         output_fp = os.path.join(output_dir, '%04d.png'%(idx))
#         util.write_img(out_seg, output_fp)

#         idx += 1

##################### zoom in
    
    focal_scale = 1.25
    focals = np.linspace(focal, focal*focal_scale, steps)
    for i, focal in enumerate(focals):
        out_img, out_seg = render_scene(model, cam_pose, _get_latent(i), focal, _OUT_SIZE)
        img_outputs.append(out_img)
        output_fp = os.path.join(output_dir, '%04d.png'%(idx))
        util.write_img(out_seg, output_fp)

        idx += 1
        

# ##################### moving x
#     _range = np.asarray([-1.0, 1.0])*0.05 / focal_scale
#     _x = np.concatenate([
#         np.linspace(0, _range[0], int(steps/2)), 
#         np.linspace(_range[0], _range[1], steps),
#         np.linspace(_range[1], 0, int(steps/2))], axis=0)

#     for i, x in enumerate(_x):
#         if rener_mode == 0:
#             cam_pose = _campos2matrix(
#                 np.array([x, 0, dpt]+cam_center), 
#                 np.array([x, 0, dpt-1]+cam_center))

#         elif rener_mode == 1:    
#             cam_pose = _campos2matrix(
#                 np.array([x*6, 0, dpt]+cam_center), 
#                 cam_center)
            
#         out_img, out_seg = render_scene(model, cam_pose, _get_latent(i, 20), focal, _OUT_SIZE)
#         img_outputs.append(out_img)

#         output_fp = os.path.join(output_dir, '%04d.png'%(idx))
#         util.write_img(out_seg, output_fp)

#         idx += 1


# ##################### moving y
#     _range = np.asarray([-0.8, 1.2])*0.05 / focal_scale
#     _y = np.concatenate([
#         np.linspace(0, _range[0], int(steps/2)), 
#         np.linspace(_range[0], _range[1], steps),
#         np.linspace(_range[1], 0, int(steps/2))], axis=0)

#     for i, y in enumerate(_y):
#         if rener_mode == 0:
#             cam_pose = _campos2matrix(
#                 np.array([0, y, dpt]+cam_center), 
#                 np.array([0, y, dpt-1]+cam_center), )

#         elif rener_mode == 1:
#             cam_pose = _campos2matrix(
#                 np.array([0, y*6, dpt]+cam_center), 
#                 cam_center.reshape((1, 3)))

#         out_img, out_seg = render_scene(model, cam_pose, _get_latent(i, 20), focal, _OUT_SIZE)
#         img_outputs.append(out_img)

#         output_fp = os.path.join(output_dir, '%04d.png'%(idx))
#         util.write_img(out_seg, output_fp)

#         idx += 1
                    
            
        ##################### Zoom Out 
#     _range = np.asarray([-1.0, 1.0])*0.3*focal + focal
#     focals = np.concatenate([
#         np.linspace(focal, _range[0], 5),
#         np.linspace(_range[0], _range[1], 20),
#         np.linspace(_range[1], focal, 5)], axis=0)

    focals = np.linspace(focal, focal/focal_scale, int(steps/2))
    for i, focal in enumerate(focals):
        out_img, out_seg = render_scene(model, cam_pose, _get_latent(i), focal, _OUT_SIZE)
        img_outputs.append(out_img)
        output_fp = os.path.join(output_dir, '%04d.png'%(idx))
        util.write_img(out_seg, output_fp)

        idx += 1
        
    ##################### Huygens    
    R = dpt
    theta = np.linspace(0, np.pi*2, num=steps*2)
    x = R * 0.4 * np.sin(theta)
    y = R * 0.2 * np.sin(theta) * np.cos(theta)
    
    for i in range(len(theta)):
        dpt = np.sqrt(R**2-x[i]**2-y[i]**2)
        cam_pose = _campos2matrix(
            np.array([x[i], y[i], dpt]+cam_center),
            cam_center)
                
        out_img, out_seg = render_scene(model, cam_pose, _get_latent(i), focal, _OUT_SIZE)
        
        img_outputs.append(out_img)
        
        output_fp = os.path.join(output_dir, '%04d.png'%(idx))
        util.write_img(out_seg, output_fp)
        
        idx += 1
    dpt = R
        
        
#     # spherical
#     steps = 30
    
#     theta = np.linspace(np.pi/2-0.2, np.pi/2+0.2, num=steps)
#     phi = np.linspace(np.pi/2-0.6, np.pi/2+0.6, num=steps)
    
#     x = R * np.sin(theta) * np.cos(phi)
#     y = R * np.sin(theta) * np.sin(phi)
#     z = R * np.cos(theta)
    
#     for i in range(steps):
#         cam_pose = _campos2matrix(
#             np.array([x[i], z[i], y[i]]+cam_center), 
#             cam_center.reshape((1, 3)))
                
#         out_img, out_seg = render_scene(model, cam_pose, _get_latent(i), focal, _OUT_SIZE)
        
#         img_outputs.append(out_img)
        
#         output_fp = os.path.join(output_dir, '%04d.png'%(idx))
#         util.write_img(out_seg, output_fp)
        
#         idx += 1
        
#     # spherical reverse
#     R = 1.1
#     steps = 30
    
#     theta = np.linspace(np.pi/2-0.2, np.pi/2+0.2, num=steps)
#     phi = np.linspace(np.pi/2+0.8, np.pi/2-0.8, num=steps)
    
#     x = R * np.sin(theta) * np.cos(phi)
#     y = R * np.sin(theta) * np.sin(phi)
#     z = R * np.cos(theta)
    
#     for i in range(steps):
#         cam_pose = _campos2matrix(
#             np.array([x[i], z[i], y[i]]+cam_center), 
#             cam_center.reshape((1, 3)))
        
#         out_img, out_seg = render_scene(model, cam_pose, _get_latent(i, inv=True), focal, _OUT_SIZE)
        
#         img_outputs.append(out_img)
        
#         output_fp = os.path.join(output_dir, '%04d.png'%(idx))
#         util.write_img(out_seg, output_fp)
        
#         idx += 1

    
#     print('output_dir = ', output_dir)    
#     imageio.mimsave(os.path.join(output_dir, 'output.gif'), img_outputs, fps=15.0)
        
#     # spiral
# #     print('*********************** SPIRAL ***********************', idx)

#     R = 0.2
#     theta = np.linspace(0, np.pi*7, num=31)
#     r = np.log(np.linspace(0, 1, num=31))*R
#     x = -r * np.sin(theta)
#     y = r * np.cos(theta)

#     plt.plot(x, y)
#     plt.show()

#     for cur_t in range(x.shape[0]-1):
#         cam_pose = _campos2matrix(
#             np.array([x[cur_t], y[cur_t], dpt]+cam_center), 
#             cam_center.reshape((1, 3)))
#         out_img, out_seg = render_scene(model, cam_pose, latent, focal, _OUT_SIZE)

#         img_outputs.append(out_img)

#     #     output_fp = os.path.join(output_dir, '%02d.png'%(idx))
#     #     util.write_img(out_img, output_fp)
#         output_fp = os.path.join(output_dir, '%04d.png'%(idx))
#         util.write_img(out_seg, output_fp)

#         idx += 1

#     output_fp = os.path.join(output_dir, '..', 'vis', os.path.basename(output_dir)+'.gif')
#     print('[DONE] save vis to : ', output_fp)
#     imageio.mimsave(output_fp, img_outputs, fps=10.0)
    
    return img_outputs, cam_pose, focal, dpt

In [5]:
########################################## 17 cls ##########################################
### ['background'0,'skin'1, 'l_brow'2, 'r_brow'2, 'l_eye'3, 'r_eye'3,'r_nose'4, 'l_nose'5, 'mouth'6, 'u_lip'7,
### 'l_lip'8, 'l_ear'9, 'r_ear'9, 'ear_r'10, 'eye_g'11, 'neck'12, 'neck_l'13, 'cloth'14, 'hair'15, 'hat'16]

# _REQUIRED_PARTS = set([0, 1,3,4,5,7,8])
# _SKIPED_PARTS = set([11])
# _NOSE_IDX = [5, 4]
############################################################################################


########################################## 20 cls ##########################################
####  ['background'0,'skin'1, 'l_brow'2, 'r_brow'3, 'l_eye'4, 'r_eye'5,'r_nose'6, 'l_nose'7, 'mouth'8, 'u_lip'9,
#### 'l_lip'10, 'l_ear'11, 'r_ear'12, 'ear_r'13, 'eye_g'14, 'neck'15, 'neck_l'16, 'cloth'17, 'hair'18, 'hat'19]

_REQUIRED_PARTS = set([0, 1, 4, 5, 6, 9, 10])
_SKIPED_PARTS = set([14])
_NOSE_IDX = [6, 7]
_DEFAULT_CAM_INT = '../checkpoints/intrinsics.txt'
############################################################################################


_CAM_K = '../checkpoints/intrinsics.txt'
_CAM_T = cam_center = np.asarray([0., 0.11, 1.3])
_CAM_CENTER = cam_center = np.asarray([0., 0.11, 0.1])


def render_sample(latent,
                  output_dir,
                  offset=0,
                  init_dpt=1.0,
                  cam_K=_CAM_K, 
                  cam_center=_CAM_CENTER,
                  validate=True):
    
    cam_int = data_util.parse_intrinsics(cam_K, trgt_sidelength=128)
    focal = cam_int[0, 0]
    cx, cy = cam_int[:2, 2]

    if _OPT_CAM:
        cam_T = (latent[:, :3]).clone().detach().squeeze().cpu().numpy()
        init_dpt = np.linalg.norm(cam_T-cam_center)
    else:
        cam_T = np.asarray([0.0, 0.0, init_dpt]) + cam_center
        
    cam_pose = _campos2matrix(cam_T, cam_center)
    
    if validate:
        out_img, out_seg = render_scene(model, cam_pose, latent, focal, _OUT_SIZE)
        all_parts = set(np.unique(out_seg)) 

        while (_SKIPED_PARTS.issubset(all_parts) or not _REQUIRED_PARTS.issubset(all_parts)):
            print('Resample latent code: ', all_parts)
            lat_idx = torch.randint(0, _TOT_NUM_INSTANCES, (1,)).squeeze().cuda()
            latent = model.get_embedding({'instance_idx': lat_idx}).unsqueeze(0)
            out_img, out_seg = render_scene(model, cam_pose, latent, focal, _OUT_SIZE)
            ##### skip images with eyeglasses
            all_parts = set(np.unique(out_seg))

        #### calculate campose from seg map
        l_nose_idx = np.asarray(np.where(out_seg==_NOSE_IDX[0]))
        r_nose_idx = np.asarray(np.where(out_seg==_NOSE_IDX[1]))
        nose_idx = np.concatenate([l_nose_idx, r_nose_idx], axis=1)        
        center = (np.mean(nose_idx[0]), np.mean(nose_idx[1]))

        yy = (256 - center[0])/(focal*4)
        xx = (center[1]-256.0)/(focal*4)

        cam_center = cam_center + np.array([xx, yy, 0.0])
        cam_T = np.array([0., 0.0, init_dpt]) + cam_center
        
        cam_pose = _campos2matrix(cam_T, cam_center)

        ############## vis ##################    
        out_img1, out_seg1 = render_scene(model, cam_pose, latent, focal, _OUT_SIZE)

        l_nose_idx = np.asarray(np.where(out_seg1==5))
        r_nose_idx = np.asarray(np.where(out_seg1==4))
        nose_idx = np.concatenate([l_nose_idx, r_nose_idx], axis=1)

        center1 = (np.mean(nose_idx[0]), np.mean(nose_idx[1]))    

        plt.axis('off')
        plt.subplot(1, 2, 1)
        plt.imshow(out_img)
        plt.subplot(1, 2, 2)
        plt.imshow(out_img1)
        plt.show()
        #####################################
        
    os.makedirs(output_dir, exist_ok=True)
    vis_outputs, cam_pose, focal, init_dpt = render_custo_path(
        0, latent, init_dpt=init_dpt, cam_center=cam_center, output_dir=output_dir, offset=offset)
    
    return vis_outputs, latent, cam_pose, focal, init_dpt, cam_center

In [7]:
# _IDX_FP = os.path.join(os.getenv("HOME"), 'liury/log/SRNs/061900seg_teaser/indexing.txt')
# with open(_IDX_FP, 'r') as f:
#     output_dirs = f.readlines()
#     output_dirs = [os.path.basename(x.strip()).split('.')[0] for x in output_dirs]
# print(output_dirs)

import random
from scipy.stats import norm

_LOG_ROOT = os.path.join(os.getenv("HOME"), 'liury/log/SRNs/test', 'custo_070821face_seg_800_imae')
os.makedirs(os.path.join(_LOG_ROOT, 'vis'), exist_ok=True)
print('Logging root = ', _LOG_ROOT)

_INTERP_STEPS = 30

# all_instances = list(range(_TOT_NUM_INSTANCES))
all_instances = list(np.loadtxt('/mnt/new_disk2/liury/log/SRNs/goodList.txt'))
for i in range(664):

#     lat_idx = random.choices(all_instances, k=2)
    lat_idx = [all_instances[i], random.choice(all_instances)]
    
    src_idx = lat_idx[0]
    trgt_idx = lat_idx[1] if lat_idx[1]!=lat_idx[0] else (lat_idx[0]+13)%_TOT_NUM_INSTANCES
    
    src_latent = model.get_embedding({'instance_idx': torch.LongTensor([src_idx]).squeeze().cuda()}).unsqueeze(0)
    trgt_latent = model.get_embedding({'instance_idx': torch.LongTensor([trgt_idx]).squeeze().cuda()}).unsqueeze(0)
        
    vis_outputs = []
    
    output_dir = os.path.join(
        _LOG_ROOT, 's%04d_t%04d'%(src_idx, trgt_idx))
    os.makedirs(os.path.join(output_dir, 'seg'), exist_ok=True)
    
    print('[%02d] %04d -> %04d: %s'%(i, src_idx, trgt_idx, output_dir))
    
    # render src-fvv 
    outputs, src_latent, cam_pose, focal, init_dpt, cam_center = render_sample(
        src_latent, output_dir, offset=len(vis_outputs), validate=False)
    vis_outputs.extend(outputs)
    
    # render interpolation
    cdf_scale = 1.0/(1.0-norm.cdf(-_INTERP_STEPS//2,0,6)*2)
    for idx in range(-_INTERP_STEPS//2,_INTERP_STEPS//2+1):
        _w = norm.cdf(idx,0,6)*cdf_scale
        latent = (1.0-_w)*src_latent + _w*trgt_latent

        out_img, out_seg = render_scene(model, cam_pose, latent, focal, _OUT_SIZE)        
        output_fp = os.path.join(output_dir, '%04d.png'%(len(vis_outputs)))
        util.write_img(out_seg, output_fp)
        vis_outputs.append(out_img)
    
    # render trgt-fvv
    outputs, _, _, _, _, _ = render_sample(
        trgt_latent, output_dir, init_dpt=init_dpt, cam_center=cam_center, 
        offset=len(vis_outputs), validate=False)
    vis_outputs.extend(outputs)
    
    vis_fp = os.path.join(_LOG_ROOT, 'vis', os.path.basename(output_dir)+'.gif')
    print('[DONE] save vis to : ', vis_fp)
    imageio.mimsave(vis_fp, vis_outputs, fps=20.0)

Logging root =  /home/anpei/liury/log/SRNs/test/custo_070821face_seg_800_imae
[00] 0000 -> 0591: /home/anpei/liury/log/SRNs/test/custo_070821face_seg_800_imae/s0000_t0591


KeyboardInterrupt: 

In [8]:
# Vis all instances

import random,os

_LOG_ROOT = os.path.join(os.getenv("HOME"), 'liury/log/SRNs/test', 'custo_070821face_seg_800_imae')
os.makedirs(os.path.join(_LOG_ROOT, 'vis'), exist_ok=True)
print('Logging root = ', _LOG_ROOT)

_INTERP_STEPS = 2
all_instances = list(range(_TOT_NUM_INSTANCES))
for i in range(1):

    lat_idx = [all_instances[i], random.choice(all_instances)]
    src_idx = all_instances[i]
    
    src_latent = model.get_embedding({'instance_idx': torch.LongTensor([src_idx]).squeeze().cuda()}).unsqueeze(0)
    
    cam_K = '../checkpoints/intrinsics.txt'
    cam_int = data_util.parse_intrinsics(cam_K, trgt_sidelength=128)
    focal = cam_int[0, 0]
    cx, cy = cam_int[:2, 2]

    cam_center = np.asarray([0., 0.11, 0.1])
    cam_T = np.asarray([0.0, 0.0, 1.0]) + cam_center
    cam_pose = _campos2matrix(cam_T, cam_center)
    print(cam_pose)


    latent = src_latent 

    out_img, out_seg = render_scene(model, cam_pose, latent, focal, _OUT_SIZE)        

    vis_fp = os.path.join(_LOG_ROOT, 'vis','%04d.png'%src_idx)
#     print('[DONE] save vis to : ', vis_fp)
    util.write_img(out_img[...,::-1], vis_fp)

Logging root =  /home/anpei/liury/log/SRNs/test/custo_070821face_seg_800_imae
[[ 1.   -0.    0.    0.  ]
 [-0.   -1.    0.    0.11]
 [ 0.   -0.   -1.    1.1 ]
 [ 0.    0.    0.    1.  ]]


In [9]:
# render with spiral path

import random,os


lookat = np.asarray([0., 0.12, 0.0])
cam_center =  np.asarray([0., 0.12, 0.97])
radii,focus_depth = [0.1,0.3,0.2],1.0 # z,x,y
# cam_poses = create_spiral_poses(radii,focus_depth)

# _LOG_ROOT = os.path.join(os.getenv("HOME"), 'liury/log/SRNs/test', 'spiral_comp', '070821face_seg_800_imae')

_LOG_ROOT = os.path.join(os.getenv("HOME"), 'liury/log/SRNs/test', 'spiral_comp', '070212face_seg_800')
os.makedirs(os.path.join(_LOG_ROOT, 'vis'), exist_ok=True)
print('Logging root = ', _LOG_ROOT)

_INTERP_STEPS = 30
# all_instances = [422,207,278,207,203,193,131,104,84,81,77,76,51,24,22,12,2]
all_instances = list(range(_TOT_NUM_INSTANCES))
# all_instances = list(np.loadtxt('/mnt/new_disk2/liury/log/SRNs/goodList.txt'))
for i in range(100):

    lat_idx = [all_instances[i], random.choice(all_instances)]
    src_idx = lat_idx[0]
    trgt_idx = (lat_idx[0]+13)%_TOT_NUM_INSTANCES
    
    src_latent = model.get_embedding({'instance_idx': torch.LongTensor([src_idx]).squeeze().cuda()}).unsqueeze(0)
    trgt_latent = model.get_embedding({'instance_idx': torch.LongTensor([trgt_idx]).squeeze().cuda()}).unsqueeze(0)
    
    vis_outputs = []
    output_dir = os.path.join(
        _LOG_ROOT, 's%04d_t%04d'%(src_idx, trgt_idx))
    os.makedirs(os.path.join(output_dir), exist_ok=True)
    
    cam_K = '../checkpoints/intrinsics.txt'
    cam_int = data_util.parse_intrinsics(cam_K, trgt_sidelength=128)
    focal = cam_int[0, 0]
    cx, cy = cam_int[:2, 2]

    # ROTATE
    n_poses=30
    R = np.linalg.norm(cam_center) + radii[0]
    theta = []
    theta_range = [0, -0.38, 0.38, 0]
    for i in range(len(theta_range)-1):
        theta.append( np.linspace(theta_range[i],theta_range[i+1], num=n_poses//4))
    theta = np.concatenate(theta)
    x = R*np.sin(theta)
    y = np.zeros_like(x)
    z = R*np.cos(theta)
    cam_T = np.stack([x,y,z],axis=1) + lookat.reshape((1,3))

    for i in range(len(theta)):
        cam_pose = _campos2matrix(cam_T[i], lookat)
        out_img, out_seg = render_scene(model, cam_pose, src_latent, focal, _OUT_SIZE)  
        vis_outputs.append(out_img)
        output_fp = os.path.join(output_dir, '%04d.png'%(len(vis_outputs)))
        util.write_img(out_seg, output_fp)
    
    # SPPIRAL PATH
#     cam_poses = create_spiral_poses(radii,focus_depth)
    t = np.linspace(0, 4*np.pi, n_poses+1)[:-1]
    for k in range(n_poses):
        cam_T = np.array([np.cos(t[k]), -np.sin(t[k]), -np.sin(0.5*t[k])]) * radii
        cam_T = cam_T[[1,2,0]] + cam_center
        cam_pose = _campos2matrix(cam_T, lookat)
        out_img, out_seg = render_scene(model, cam_pose, src_latent, focal, _OUT_SIZE)  
        vis_outputs.append(out_img)
        output_fp = os.path.join(output_dir, '%04d.png'%(len(vis_outputs)))
        util.write_img(out_seg, output_fp)

    cdf_scale = 1.0/(1.0-norm.cdf(-_INTERP_STEPS//2,0,6)*2)
    for idx in range(-_INTERP_STEPS//2,_INTERP_STEPS//2+1):
        _w = norm.cdf(idx,0,6)*cdf_scale
        latent = (1.0-_w)*src_latent + _w*trgt_latent
        out_img, out_seg = render_scene(model, cam_pose, latent, focal, _OUT_SIZE) 
        vis_outputs.append(out_img)
        output_fp = os.path.join(output_dir, '%04d.png'%(len(vis_outputs)))
        util.write_img(out_seg, output_fp)
        
    for k in range(n_poses):
        cam_T = np.array([np.cos(t[k]), -np.sin(t[k]), -np.sin(0.5*t[k])]) * radii
        cam_T = cam_T[[1,2,0]] + cam_center
        cam_pose = _campos2matrix(cam_T, lookat)
        out_img, out_seg = render_scene(model, cam_pose, trgt_latent, focal, _OUT_SIZE)  
        vis_outputs.append(out_img)
        output_fp = os.path.join(output_dir, '%04d.png'%(len(vis_outputs))) 
        util.write_img(out_seg, output_fp)

    for idx in range(-_INTERP_STEPS//2,_INTERP_STEPS//2+1):
        _w = norm.cdf(idx,0,6)*cdf_scale
        latent = (1.0-_w)*trgt_latent + _w*src_latent
        out_img, out_seg = render_scene(model, cam_pose, latent, focal, _OUT_SIZE) 
        vis_outputs.append(out_img)
        output_fp = os.path.join(output_dir, '%04d.png'%(len(vis_outputs)))
        util.write_img(out_seg, output_fp)
        
    vis_fp = os.path.join(_LOG_ROOT, 'vis', os.path.basename(output_dir)+'.gif')
    print('[DONE] save vis to : ', vis_fp)
    imageio.mimsave(vis_fp, vis_outputs, fps=5.0)

Logging root =  /home/anpei/liury/log/SRNs/test/spiral_comp/070212face_seg_800
[DONE] save vis to :  /home/anpei/liury/log/SRNs/test/spiral_comp/070212face_seg_800/vis/s0000_t0013.gif
[DONE] save vis to :  /home/anpei/liury/log/SRNs/test/spiral_comp/070212face_seg_800/vis/s0001_t0014.gif
[DONE] save vis to :  /home/anpei/liury/log/SRNs/test/spiral_comp/070212face_seg_800/vis/s0002_t0015.gif
[DONE] save vis to :  /home/anpei/liury/log/SRNs/test/spiral_comp/070212face_seg_800/vis/s0003_t0016.gif
[DONE] save vis to :  /home/anpei/liury/log/SRNs/test/spiral_comp/070212face_seg_800/vis/s0004_t0017.gif
[DONE] save vis to :  /home/anpei/liury/log/SRNs/test/spiral_comp/070212face_seg_800/vis/s0005_t0018.gif
[DONE] save vis to :  /home/anpei/liury/log/SRNs/test/spiral_comp/070212face_seg_800/vis/s0006_t0019.gif
[DONE] save vis to :  /home/anpei/liury/log/SRNs/test/spiral_comp/070212face_seg_800/vis/s0007_t0020.gif
[DONE] save vis to :  /home/anpei/liury/log/SRNs/test/spiral_comp/070212face_seg_

In [124]:
cam_pose 

array([[ 1.       , -0.       ,  0.       ,  0.       ],
       [-0.       , -1.       ,  0.       ,  0.12     ],
       [ 0.       , -0.       , -1.       ,  1.0773945],
       [ 0.       ,  0.       ,  0.       ,  1.       ]])

In [ ]:
_DEFAULT_CAM_INT = '../checkpoints/intrinsics.txt'
_LOG_ROOT = os.path.join(os.getenv("HOME"), 'liury/log/SRNs')

cam_int = data_util.parse_intrinsics(_DEFAULT_CAM_INT, trgt_sidelength=128)
focal = cam_int[0, 0]
cam_center = np.array([0,0.11,0.1])


for i in range(150):
    lat_idx = torch.randint(0, _TOT_NUM_INSTANCES, (2,)).squeeze().cuda()
    src_latent = model.get_embedding({'instance_idx': lat_idx[0]}).unsqueeze(0)
    trgt_latent = model.get_embedding({'instance_idx': lat_idx[1]}).unsqueeze(0)
    
    src_idx = lat_idx[0].cpu().numpy()
    trgt_idx = lat_idx[1].cpu().numpy()
        
    output_dir = os.path.join(
        _LOG_ROOT, 'interp_latent', 's%04d_t%04d'%(src_idx, trgt_idx))
    os.makedirs(os.path.join(output_dir, 'seg'), exist_ok=True)
    
    print('[%02d] %04d -> %04d: %s'%(i, src_idx, trgt_idx, output_dir))
    
    R = np.random.rand()*0.5 + 0.7
    theta = np.random.rand()*0.4 + (np.pi/2-0.2)
    phi = np.random.rand()*1.2 + (np.pi/2-0.6)
    
    x = R * np.sin(theta) * np.cos(phi)
    y = R * np.sin(theta) * np.sin(phi)
    z = R * np.cos(theta)
    
    cam_pose = _campos2matrix(np.array([x, z, y])+cam_center, cam_center)
    
    steps = 30
    
    img_outputs = []    
    
    for idx in range(steps):
        _w = float(idx) / (steps - 1)
        latent = (1.0-_w)*src_latent + _w*trgt_latent
        
        out_img, out_seg = render_scene(model, cam_pose, latent, focal, _OUT_SIZE)
        
        img_outputs.append(out_img)
        output_fp = os.path.join(output_dir, '%04d.png'%(idx))
        util.write_img(out_seg, output_fp)
        
    imageio.mimsave(os.path.join(output_dir, 'output.gif'), img_outputs, fps=10.0)

In [27]:
# load latent
from glob import glob
from torchvision.utils import make_grid
import torch.nn.functional as F
import cv2

from dataset import data_util
from dataset.data_util import load_seg_map

_ORI_DATA_ROOT = '/mnt/new_disk2/liury/data/facial-data/CelebAMask-HQ/segmap_20'
_LOG_ROOT = os.path.join(os.getenv("HOME"), 'liury/log/SRNs')
_LATENT_ROOT = os.path.join(_LOG_ROOT, '060115face_celebA/latent_codes')

all_latents = glob(os.path.join(_LATENT_ROOT, '*.npy'))

print('tot latents: ', len(all_latents))

_DEFAULT_CAM_INT = '../checkpoints/intrinsics.txt'
cam_int = data_util.parse_intrinsics(_DEFAULT_CAM_INT, trgt_sidelength=128)
focal = cam_int[0, 0]
cam_center = np.array([0,0.11,0.0])

for i in range(27, len(all_latents)):
    out_imgs = []    
    
    item_id = int(os.path.basename(all_latents[i]).split('.')[0].split('_')[1])
    latent = np.load(all_latents[i])
    
    # load ori
    ori_seg = torch.from_numpy(load_seg_map(os.path.join(_ORI_DATA_ROOT, '%d.png'%(item_id)), _OUT_SIZE))
    ori_seg_img = util.lin2img(ori_seg.unsqueeze(0), color_map=_CMAP).cpu().numpy()
    ori_seg_img = (ori_seg_img.squeeze().transpose(1, 2, 0)) * 255.0
    ori_seg_img = ori_seg_img.round().clip(0, 255).astype(np.uint8)
    out_imgs.append(torch.from_numpy(ori_seg_img).permute(2, 0, 1).unsqueeze(0))
        
    # predict
    cam_T = latent[:3]
    latent = torch.from_numpy(latent[3:]).unsqueeze(0).cuda()
    R = np.linalg.norm(cam_T)

    cam_pose = _campos2matrix(cam_T+cam_center, cam_center)
    out_img, out_seg = render_scene(model, cam_pose, latent, focal, _OUT_SIZE)
    out_imgs.append(torch.from_numpy(out_img).permute(2, 0, 1).unsqueeze(0))
    
    # calc mIoU
    ori_seg = F.one_hot(ori_seg.squeeze().long(), 20)
    out_seg = F.one_hot(torch.from_numpy(out_seg).flatten().long(), 20)
    
    print(ori_seg.shape, out_seg.shape)
    break
    
    mIoU = torch.mean(torch.div(
        torch.sum(ori_seg&out_seg, dim=0).float()+1e-8,
        torch.sum(ori_seg|out_seg, dim=0).float()+1e-8))
                
    for step in range(6):
    
        theta = np.random.rand()*0.4 + (np.pi/2-0.2)
        phi = np.random.rand()*1.2 + (np.pi/2-0.6)

        x = R * np.sin(theta) * np.cos(phi)
        y = R * np.sin(theta) * np.sin(phi)
        z = R * np.cos(theta)

        cam_pose = _campos2matrix(np.array([x, z, y])+cam_center, cam_center)
        out_img, out_seg = render_scene(model, cam_pose, latent, focal, _OUT_SIZE)
                
        out_imgs.append(torch.from_numpy(out_img).permute(2, 0, 1).unsqueeze(0))
    
        
    out_imgs = make_grid(torch.cat(out_imgs), nrow=8, padding=1).permute((1, 2, 0)).cpu().numpy()
    
    ############## vis ##################    
    fig = plt.figure(figsize=(45, 5))
    plt.imshow(out_imgs)
    plt.axis('off')
    plt.show()
    ####################################
    
    output_dir = os.path.join(_LOG_ROOT, 'change_view_celebA', '%06d'%(item_id))
    os.makedirs(os.path.join(output_dir, 'seg'), exist_ok=True)    
    render_custo_path(0, latent, cam_center=cam_center, output_dir=output_dir)
    
    print('[%06d] %d : R = %f, mIoU = %f, output_dir = %s'%(i, item_id, R, mIoU, output_dir))

tot latents:  80
torch.Size([262144, 20]) torch.Size([262144, 20])


In [ ]:
aa = torch.cat()

In [ ]:
for i in range(100):
    lat_idx = torch.randint(0, _TOT_NUM_INSTANCES, (2,)).squeeze().cuda()
    src_latent = model.get_embedding({'instance_idx': lat_idx[0]}).unsqueeze(0)
    trgt_latent = model.get_embedding({'instance_idx': lat_idx[1]}).unsqueeze(0)
    
    src_idx = lat_idx[0].cpu().numpy()
    trgt_idx = lat_idx[1].cpu().numpy()
    
    print('%02d: %04d -> %04d'%(i, src_idx, trgt_idx))
    
    output_dir = os.path.join(
        _LOG_ROOT, 'out_mode_3', 's%04d_t%04d'%(src_idx, trgt_idx))
    os.makedirs(os.path.join(output_dir, 'seg'), exist_ok=True)
    
    print(output_dir)
    
    render_custo_path(0, src_latent, trgt_latent, output_dir=output_dir)

    cam_pose = _campos2matrix(np.array([0., 0., 0.8])+cam_center, cam_center)
    out_img, out_seg = render_scene(model, cam_pose, _get_latent(i), focal, _OUT_SIZE)

In [ ]:
from dataset.face_dataset import FaceRandomPoseDataset
from torch.utils.data import DataLoader
# from torch.utils.tensorboard import SummaryWriter

import cv2
import imageio

_OUTPUT_DIR = os.path.join(_LOG_ROOT, 'output_iter_020000')
_MODE = 'sphere'
_R = 1.5


_NUM_INSTANCES=10
_NUM_OBSERVATIONS=25

output_dir = os.path.join(_OUTPUT_DIR, _MODE+'_128')
sample_instances = list(np.random.choice(range(_TOT_NUM_INSTANCES), _NUM_INSTANCES, replace=False))

# writer = SummaryWriter(output_dir)

dataset = FaceRandomPoseDataset(
    intrinsics=_CAM_INT,
    cam_center=_CAM_CENTER,
    num_instances=sample_instances, 
    num_observations=_NUM_OBSERVATIONS, 
    sample_radius=_R, mode=_MODE)

dataloader = DataLoader(dataset,
                     collate_fn=dataset.collate_fn,
                     batch_size=1,
                     shuffle=False,
                     drop_last=False)

print('Beginning evaluation...')

images = {}

with torch.no_grad():

    for idx, model_input in enumerate(dataloader):
        model_input, ground_truth = model_input
        
        pose = model_input['pose']
        
#         print(pose)
        
        intrinsics = model_input['intrinsics']
        uv = model_input['uv']
        z = model.get_embedding(model_input)
        
        model_outputs = model(pose, z, intrinsics, uv)
        predictions, depth_maps = model_outputs
                
        batch_size, tensor_len, channels = predictions.shape
        img_sidelen = np.sqrt(tensor_len).astype(int)
                
        pred = torch.argmax(predictions, dim=2, keepdim=True)
        output_img = util.lin2img(pred, color_map=).cpu().numpy()
        output_pred = pred.view(batch_size, img_sidelen, img_sidelen, 1).cpu().numpy()

        for i in range(output_img.shape[0]):
            instance_idx = int(model_input['instance_idx'][i].squeeze().detach().cpu().numpy().astype(np.int64))
            observation_idx = model_input['observation_idx'][i]
            
            instance_dir = os.path.join(output_dir, "%03d" % instance_idx)
            os.makedirs(instance_dir, exist_ok=True)
            
            img = output_img[i, :, :, :].squeeze().transpose(1, 2, 0)
            img *= 255
            img = img.round().clip(0, 255).astype(np.uint8)
#             img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 

            if not instance_idx in images:
                images[instance_idx] = [None] * _NUM_OBSERVATIONS

            images[instance_idx][observation_idx] = img
#             output_fp = os.path.join(instance_dir, '%02d.png'%(observation_idx))
#             util.write_img(img, output_fp)
            
            output_fp = os.path.join(instance_dir, '%02d_seg.png'%(observation_idx))
            seg_img = output_pred[i, :, :].squeeze().astype(np.uint8)
            
            util.write_img(seg_img, output_fp)
            
#             dpt_img = (output_dpt[i, :, :, :].squeeze() * 255.0).astype(np.uint8)
#             dpt_img = cv2.applyColorMap(dpt_img, cv2.COLORMAP_JET)
#             output_fp = os.path.join(instance_dir, '%02d_depth.png'%(observation_idx))
# #             print('Save output for instance %03d - %02d: %s'%(instance_idx, observation_idx, output_fp))
#             util.write_img(dpt_img, output_fp)
            print('Save output for instance %04d - %02d: %s'%(instance_idx, observation_idx, output_fp))
        
            if observation_idx == _NUM_OBSERVATIONS - 1:
                imageio.mimsave(os.path.join(instance_dir, 'output.gif'), images[instance_idx], fps=5.0)
                print('=== [DONE] saving output.gif.')
            
#         print('[DONE] Save output for instance %03d.'%(instance_idx))


# misc

In [ ]:
# move seg_face_8000
import os
from glob import glob
from shutil import copyfile

data_root = '/data/anpei/facial-data/seg_face_8000/images'
img_fps = glob(os.path.join(data_root, '*_seg_*.png'))
img_ids = list(set([os.path.basename(x)[:5] for x in img_fps]))

for img_id in img_ids:
    output_dir = os.path.join(data_root, img_id)
    os.makedirs(output_dir, exist_ok=True)
    copyfile(
        os.path.join(data_root, '..', 'cam2world.npy'), 
        os.path.join(output_dir, 'cam2world.npy'))
    
    img_fps = glob(os.path.join(data_root, '%s_seg_*.png'%(img_id)))
    print(output_dir, len(imgs))
    for img_fp in sorted(img_fps):
        out_name = os.path.basename(img_fp).split('_')
        out_sub_dir = out_name[0]
        out_name = '_'.join([out_name[1], '%02d'%(int(out_name[-2])*5+int(out_name[-1][:2]))]) + '.png'
        out_name = os.path.join(os.path.dirname(img_fp), out_sub_dir, out_name)
#         print('\t> ', img_fp, out_name)
        os.rename(img_fp, out_name)

In [ ]:
import os
import numpy as np

data_dir = '/data/anpei/facial-data/seg_face_2000/10996'

cam_param = np.load(os.path.join(data_dir, 'cameras.npy'), allow_pickle=True).item() 
print(np.asarray(cam_param['zRange']).shape, np.asarray(cam_param['extrinsics']).shape)

In [ ]:
import torchvision.models as torch_models
import torch

resnet_model = torch_models.resnet18(pretrained=False)
# resnet_model.layer4 = torch.nn.Identity()
# resnet_model.fc = torch.nn.Identity()
resnet_model.conv1 = torch.nn.Conv2d(19, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
print(resnet_model)

In [ ]:
a = resnet_model(torch.Tensor(8, 19, 128, 128))
print(a.shape)

In [ ]:
import torch
from torch.nn import functional as F

aa = torch.randint(0, 19, size=(1, 3, 3))
print(aa.shape)

bb = F.one_hot(aa)
print(bb.shape, bb[0])

In [ ]:
aa = torch.empty(256)
print(aa.shape)

In [ ]:
import cv2

instance_dir = '/data/anpei/facial-data/seg_body/94'
seg_imgs = img = cv2.imread(os.path.join(instance_dir, 'semantic_mask', 'image.cam05_000094.png'), cv2.IMREAD_UNCHANGED).astype(np.float32)
print(np.unique(seg_imgs))


fs = cv2.FileStorage("/data/anpei/facial-data/seg_body/Calib_T_samba/cam05_000000/intrinsic.xml", cv2.FILE_STORAGE_READ)
intrinsics = fs.getNode("M").mat()

print(intrinsics)

H, W = seg_imgs.shape
cx, cy = intrinsics[:2, 2]

x_coord, y_coord = np.where(seg_imgs != 0)
bbox = np.asarray([np.min(x_coord), np.min(y_coord), np.max(x_coord), np.max(y_coord)])
print(bbox)

sidelength = max(bbox[2] - bbox[0], bbox[3] - bbox[1])
center = np.asarray([bbox[2] + bbox[0], bbox[3] + bbox[1]]) / 2.0
bbox = np.ceil(np.asarray([
    center[0]-sidelength/2.0, 
    center[1]-sidelength/2.0, 
    center[0]+sidelength/2.0,
    center[1]+sidelength/2.0])).astype(np.int64)

cx, cy = intrinsics[:2, 2] - bbox[:2]

print(bbox, bbox.dtype)
seg_imgs = seg_imgs[bbox[0]:bbox[2], bbox[1]:bbox[3]]
print(seg_imgs.shape)


print(cx, cy, bbox)
intrinsics[:2, 2] = [cx, cy]
print(intrinsics)


print(H, W, cx, cy)

In [ ]:
from dataset.body_dataset import BodyPartDataset
from torch.utils.data import DataLoader
import numpy as np

import util

from matplotlib import pyplot as plt

dataset = BodyPartDataset(
        data_type='seg',
        img_sidelength=128, 
        sample_observations=list(range(5)),
        sample_instances=list(range(2)))

dataloader = DataLoader(dataset,
                     collate_fn=dataset.collate_fn,
                     batch_size=1,
                     shuffle=False,
                     drop_last=False)

print(len(dataloader))

for idx, model_input in enumerate(dataloader):
    model_input, ground_truth = model_input
        
    seg_img = model_input['rgb']
    pose = model_input['pose']
    intrinsics = model_input['intrinsics']
    uv = model_input['uv']    
    
    print('> ', np.unique(seg_img), seg_img.shape)
    print('> pose = ')
    print(pose)
    print('> cam_int = ')
    print(intrinsics)
    print(seg_img.shape, np.unique(seg_img))
    
    output_img = util.lin2img(seg_img, color_map=dataset.color_map).cpu().numpy()
    
    plt.imshow(output_img.squeeze().transpose(1, 2, 0))
    plt.show()

In [ ]:
import torchvision.models as models

model = models.resnet18(pretrained=False)
print(model)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

H, W = 512, 512


class ConvBlock(nn.Module):
    def __init__(   self, 
                    in_channels, 
                    out_channels, 
                    kernel_size=3, 
                    stride=1, 
                    num_groups=2):
        super(ConvBlock, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride, stride//2),
            nn.GroupNorm(num_groups, out_channels),
            nn.ReLU())

    def forward(self, x):
        return self.conv(x)
    

class FeatureExtractor(nn.Module):
    def __init__(self, in_channels=3):
        super(FeatureExtractor, self).__init__()
        self.in_channels = in_channels
        
        self.conv1_1 = ConvBlock(in_channels, 8, 3, 1)
        self.conv1_2 = ConvBlock(8, 16, 3, 2)
        self.conv2_1 = ConvBlock(16, 16, 3, 1)
        self.conv2_2 = ConvBlock(16, 32, 3, 2)
        self.conv3_1 = ConvBlock(32, 32, 3, 1)
        self.deconv4_1 = nn.ConvTranspose2d(32, 16, 3, 2)

        self.conv4_2 = ConvBlock(32, 16, 3, 1)
        self.deconv5_1 = nn.ConvTranspose2d(16, 8, 3, 2)

        self.conv5_2 = ConvBlock(16, 8, 3, 1)

    def forward(self, x):
        x0 = self.conv1_1(x)
        print(x0.shape)
        x = self.conv1_2(x0)
        x1 = self.conv2_1(x)
        x = self.conv2_2(x)
        out0 = self.conv3_1(x)
        x = self.deconv4_1(out0)

        x = torch.cat([x1, x], dim=1)
        out1 = self.conv4_2(x)
        x = self.deconv5_1(out1)
        x = torch.cat([x0, x], dim=1)

        out2 = self.conv5_2(x)
        return out0, out1, out2

In [ ]:
fake_in = torch.rand(8, 3, H, W)
print(fake_in.shape)

feat_worker = FeatureExtractor()

output = feat_worker(fake_in)

print(output.shape)

In [ ]:
import os
from image_utils import Random_Transforms

basedir='/mnt/wmy/sport_1_mask'
cam_id=0

poses = np.loadtxt(os.path.join(basedir,'CamPose.inf'))
Ks = read_intrinsics(os.path.join(basedir,'Intrinsic.inf'))

img_fp = os.path.join(basedir, 'img', '%d/img_%04d.jpg' % (_FRAME_ID, cam_id))
img = imageio.imread(img_fp)
mask_fp = os.path.join(basedir, 'img', 'mask', '%d/img_%04d.jpg' % (_FRAME_ID, cam_id))
mask = imageio.imread(mask_fp)

img, K, img_mask, ROI = transform(img, Ks[cam_id], mask)

print(img.shape)

print(K)

In [ ]:
import imageio

img_fp = os.path.join(basedir, 'img', '%d/img_%04d.jpg' % (0, 0))
img = imageio.imread(img_fp)

print(img.shape)

In [ ]:
print(Ks[0, 0, 0])

In [ ]:
jupyter nbextension enable --py widgetsnbextension

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
interact(f, x='Hi there!');

In [ ]:
data_dir = 